In [ ]:
# |default_exp pingme_class
# You need this at the top of every notebook you want turned into a module, the name your provide will determine the module name

Now that you've finished the getting started in `./GETTING_STARTED_WITH_TEMPLATE.md`, you'll notice the directory has many new files and folders. In this example `$YOUR_REPO_NAME` is `template_nbdev_example` so be sure to adjust accordingly.

There's now 
```bash
./_docs
.quarto
./template_nbdev_example # This is the code autogenerated from the notebooks, you should only adjust this through your notebooks
./template_nbdev_example/__pycache__
./template_nbdev_example/__init__.py
./template_nbdev_example/_modidx.py
./template_nbdev_example/core.py
./template_nbdev_example/hello_world.py
./template_nbdev_example.egg-info # This is metadata about the package for pip
_quarto.yml
index.ipynb # This is a notebook for showing how the program works and generated the README.md, you should adjust this
MANIFEST.in # This determines what files outside of .py files are included in the package, you may need to adjust it.
README.md
setup.py # This is the file that tells pip how to install the package, you shouldn't need to edit this ever
styles.css
```

>If you want to check what you're documentation looks like run `nbdev_preview` in command line of the project folder

# Additional Learning Resources
Make sure you check out [Code Standards](https://dksund.sharepoint.com/:fl:/g/contentstorage/CSP_7c761ee7-b577-4e08-8517-bc82392bf65e/EVddgBNVhOxJm8q27-KyX-UBdfPBbcqtjMK1DquKhDUPiA?e=E8uw7f&nav=cz0lMkZjb250ZW50c3RvcmFnZSUyRkNTUF83Yzc2MWVlNy1iNTc3LTRlMDgtODUxNy1iYzgyMzkyYmY2NWUmZD1iJTIxNXg1MmZIZTFDRTZGRjd5Q09TdjJYblkwVlNiWXFYcE1yaHVrVmZqTVJUVEE4X1VwZjhTd1JxcjRNdmFrSmh2RCZmPTAxVlVLVzVWS1hMV0FCR1ZNRTVSRVpYU1ZXNTdSTEVYN0YmYz0lMkYmYT1Mb29wQXBwJnA9JTQwZmx1aWR4JTJGbG9vcC1wYWdlLWNvbnRhaW5lciZ4PSU3QiUyMnclMjIlM0ElMjJUMFJUVUh4a2EzTjFibVF1YzJoaGNtVndiMmx1ZEM1amIyMThZaUUxZURVeVpraGxNVU5GTmtaR04zbERUMU4yTWxodVdUQldVMkpaY1Zod1RYSm9kV3RXWm1wTlVsUlVRVGhmVlhCbU9GTjNVbkZ5TkUxMllXdEthSFpFZkRBeFZsVkxWelZXU1RJMVJsaFBNalkyUlZkQ1FqTTFRVmhKVTBkRFVVcFdXa1klM0QlMjIlMkMlMjJpJTIyJTNBJTIyNGQ4MWUyZGQtMTUyMy00Y2U1LTg3OWItZWU1ZTNhYWUyOThiJTIyJTdE) (<b>Recommended</b> requires SSI login)

For some markdown info use [markdown basic syntax](https://www.markdownguide.org/basic-syntax/)

To run through the nbdev tutorial [nbdev documentation](https://nbdev.fast.ai/tutorial.html), try and see all the functions they offer

And to see this example fully fleshed out [nbdev_tutorial_example](https://github.com/ssi-dk/template-nbdev-example), the result of using this template and following the remaining steps.

In [ ]:
# |hide
# See above? this hides these blocks, meaning these blocks aren't in the module and aren't in the documentation
import nbdev
from nbdev.showdoc import *  # ignore this Pylance warning in favor of following nbdev docs

# Libraries
Here we include all the libraries of this module. You can see they're sectioned so the top parts can be easy cut and paste into new files.

In [ ]:
# |export
# That export there, it makes sure this code goes into the module.

# standard libs
import os
import json  # to manage json payloads
import re  # regular expression for parsing

# Common to template
# add into settings.ini, requirements, package name is python-dotenv, for conda build ensure `conda config --add channels conda-forge`
import fastcore  # To add functionality related to nbdev development, https://github.com/fastai/fastcore/
from fastcore.script import (
    call_parse,
)  # for @call_parse, https://fastcore.fast.ai/script
from fastcore import test
from fastcore.utils import patch

# Project specific libraries
from pydantic import BaseModel

from pingme import (
    core,
)
import sys
from sys import stderr

import email.mime.text  # to format emails
import smtplib
import datetime

from pingme.core import settings

Normally your imports go into Project specific libraries above, but we'll put it in a code block here. In this example you'll want to comment out the code below, because YOUR_REPO_NAME changes with each repository, it'll cause issues if you try to run it with a different repository name

Because the notebooks now are located in the `nbs` folder, we need to change the python `wd` for the notebook to the project folder. Keep this included in all notebooks but don't export it to the package. 

In [ ]:
# This block should never be exported. It is to have python running in the project (and not the nbs) dir, and to initiate the package using pip.
os.chdir(core.PROJECT_DIR)

Now you have access to your functions in core.py and call call them here.

NOTE: if you change another notebook, run nbdev_prepare, and restart your current kernel to see the changes

Here we'll load the config file values, note that the file isn't exported so is for development and documentation purposes.  
  
>**A note on config files**: The final package should only contain `config.default.*` files. These files are located in *package dir* `PACKAGE_DIR/config`. For development, custom config files can be specified in the *project dir* `PROJECT_DIR/config/` folder. These files will not be shipped with the package, but for development, they can be accessed from the project dir (thanks to to code above), which usually is your working directory in terminal while developping.  

Lets look at our values, as we have a dictionary, that can be viewed more nicely as a json object

In [ ]:
# | export
from pydantic import BaseModel


class Card(BaseModel):
    name: str
    context: dict

In [ ]:
# |export
# Imports at top
# import json  # to manage json payloads
# import re  # regular expression for parsing
@staticmethod
def resolved_payload(template: json, context: dict) -> json:
    """
    Resolves the payload by substituting variables in the `payload` with values from the `context` and ensures all variables are accounted for

    Args:
    template: json, the payload to be resolved
    context: dict, the values to substitute into the payload

    Returns:
    json, the resolved payload
    """
    if template is None:
        # Ensure there is a payload
        raise ValueError("Payload is None")
    str_temp = json.dumps(template)  # convert payload to string
    for key in context.keys():
        # Substitute all variables in payload with values from payload_context, it can also be set up that their are no variables in the payload
        str_temp = str_temp.replace("${" + key + "}", context[key])
    if re.search("${.*}", str_temp):
        # Check if there are any variables left, this is not allowed
        raise ValueError("Unresolved variables in payload")
    return json.loads(str_temp)

In [ ]:
# |export


class PingMe:
    """
    PingMe class which notifies via either a webhook or email
    """

    def __init__(self, card: Card, config_file=None):  # Extension of card file
        """
        Initializes the PingMe object
        Args:
        card: Card, the card to be sent
        config_file: str, the path to the config file
        """

        # Resolve config variables from ENV vars
        if config_file is None:
            config_file = "./config/example.env"
        config = core.get_config(os.environ.get("CORE_CONFIG_FILE", config_file))
        if card.name not in config["pingme"]["cards"]:
            raise ValueError(
                f"Card name {card.name} not found in config file, check spelling"
            )
        self.card: dict = {}
        self.card = config["pingme"]["cards"][card.name]
        self.card["context"] = card.context

        # Set default values for card variables if not provided in context
        for item in self.card["variables"]:
            if item not in self.card["context"]:
                self.card["context"][item] = self.card["variables"][item]

        # Get title and text which are special variables
        self.title: str = self.card["context"].get("title", "")
        self.text: str = self.card["context"].get("text", "")

        # Set options
        self.email: dict = config["pingme"]["options"]["email"]
        self.webhook: dict = config["pingme"]["options"]["webhook"]
        self.logfile: dict = config["pingme"]["options"]["logfile"]

        # Resolve payload variables from card.context, defined below
        self.payload: json = resolved_payload(
            self.card["template"], self.card["context"]
        )

    def __str__(self) -> str:
        return f"""PingMe object with:
    card: {self.card}
    card_context: {self.card["context"]}
    payload: {self.payload}"""

    def __repr__(self) -> str:
        return self.__str__()

In [ ]:
# Test
config = core.get_config("./config/config.env")
card = Card.model_validate(
    {
        "name": config["pingme"]["user_input"]["card"]["name"],
        "context": config["pingme"]["user_input"]["card"]["context"],
    }
)
pingme = PingMe(card, config_file="./config/config.env")
pingme.payload

Lets add a test here as well, which will get run through `./.github/workflows/test.yaml` whenever changes happen to the repository

In [ ]:
# |export
import requests  # to send requests to webhooks


@staticmethod
def send_to_webhook(
    url: str, payload: json, header: json = {"Content-Type": "application/json"}
) -> json:
    """
    Sends a message to a webhook

    Args:
    url: str, the webhook URL
    payload: json, the payload to be sent
    header: json, the header to be sent

    Returns:
    json, the response from the webhook
    """
    if url is None:
        raise Exception("Webhook URL not set")
    # Send message to webhook
    try:
        response = requests.post(url, data=payload, headers=header)
    except Exception as e:
        raise Exception(f"Error sending message to webhook: {e}")
    return response

In [ ]:
# Test
with fastcore.test.ExceptionExpected():
    send_to_webhook("https://badhost", json.dumps(pingme.payload))

In [ ]:
# |export
@patch
def send_webhook(self: PingMe) -> dict:
    return send_to_webhook(self.webhook["url"], json.dumps(self.payload))

In [ ]:
pingme.webhook

In [ ]:
response = requests.post(
    pingme.webhook["url"],
    data=json.dumps(pingme.payload),
    headers={"Content-Type": "application/json"},
)
# empty response is normal
print(response.content)

In [ ]:
# |export
# Imports at the top
# import email.mime.text  # to format emails
# import smtplib  # to send emails
@staticmethod
def send_to_email(
    payload: json,
    subject: str,
    from_: str,
    to: str,
    host: str,
    port: int = 25,
    user=None,
    password=None,
) -> dict:
    """
    Sends a message to an email address

    Args:
    payload: json, the payload to be sent
    subject: str, the subject of the email
    from_: str, the sender of the email
    to: str, the recipient of the email
    host: str, the host of the email server
    port: int, the port of the email server
    user: str, the username of the email server
    password: str, the password of the email server

    Returns:
    dict, the response from the email server

    # NOTE: Wondering if I should do something more like https://learn.microsoft.com/en-us/graph/api/user-sendmail?view=graph-rest-1.0&tabs=http
    """
    email_status = False
    html_content = """






            This is a sample body


    """
    msg = email.mime.text.MIMEText(html_content, "html")
    msg["Subject"] = subject
    msg["From"] = from_
    msg["To"] = to
    email_connection = smtplib.SMTP(host, port)
    try:
        email_connection.ehlo()
        email_connection.starttls()
        email_connection.ehlo()
        email_connection.login(user, password)
        email_connection.sendmail(from_, to, msg.as_string())
        email_status = True
    finally:
        email_connection.quit()
        return json.dumps({"response": email_status})

In [ ]:
# |export
@patch
def send_email(self: PingMe) -> dict:
    return send_to_email(
        self.payload,
        self.title,
        self.email["from"],
        self.email["to"],
        self.email["smtp"]["host"],
        self.email["smtp"]["port"],
        self.email["smtp"]["user"],
        self.email["smtp"]["password"],
    )

In [ ]:
# |export
# import datetime  # to get current date and time which is used in logging
@staticmethod
def send_to_logfile(logfile: str, title: str, text: str) -> dict:
    """
    Send message to logfile

    Args:
    logfile: str, the path to the logfile
    title: str, the title of the message
    text: str, the text of the message

    Returns:
    dict, the response from the logfile

    TODO: The log file only logs title and text right now (not payload), while payload can be included it is not good for parsing. Need to think of a solution for this. Could be to save each payload as a seperate file and the log is a list of files.
    """
    if logfile is None:
        raise Exception("Log file not set")
    with open(logfile, "a") as f:
        # Write the current time
        f.write(f"{datetime.datetime.now()}\t{title}\t{text}\n")
        f.write("\n")
    return json.dumps({"status_code": 200, "response": True})

In [ ]:
# |export
@patch
def send_logfile(self: PingMe) -> dict:
    return send_to_logfile(self.logfile["path"], self.title, self.text)

In [ ]:
# |export
# Make a CLI function using `call_parse` to handle arguments
# Ensure settings.ini contains `console_scripts = pingme=pingmeme:cli`, this makes the call as `pingme` and calls the cli function found in package pingme.pingme
@call_parse
def cli(
    context: str = None,  # string denoting a json object with context variables (e.g. '{"title":"Test Title", "text":"Test Text"}')
    webhook: bool = None,  # attempts to send to webhook
    email: bool = None,  # attempts to send to email
    logfile: bool = None,  # attempts to send to logfile
    example: bool = None,  # Runs with example params, if it doesn't work config values haven't been set properly
    config_file: str = None,  # config file to set env vars from
):
    """
    PingMe send a notification to a webhook, email, or log file.\n\n
    Usage examples:
    - basic:
    pingme --context '{"title":"Test Title", "text":"Test Text"}' --webhook
    - advanced:
    pingme --config_file ./config/config.env --context '{"title":"Test Title", "text":"Test Text"}' --webhook --email --logfile --card_name default --card_dir ./cards/ --card_ext .yaml
    NOTE: Will require use of ./cards/default.yaml and ./config/config.default.env to be set up properly

    Returns:
    bool, True if successful, False if not
    """
    config = core.get_config(config_file)

    card = Card
    card.name = config["pingme"]["user_input"]["card"]["name"]
    card.context = config["pingme"]["user_input"]["card"]["context"]
    print(card)
    pingme = PingMe(card, config_file)

    if not webhook and not email and not logfile:
        print("No destination provided, exiting", file=sys.stderr)
        sys.exit(1)
    else:
        if webhook:
            pingme.send_webhook()
            print("Sent to webhook", file=sys.stdout)
        if email:
            pingme.send_email()
            print("Sent to email", file=sys.stdout)
        if logfile:
            pingme.send_logfile()
            print("Sent to logfile", file=sys.stdout)

In [ ]:
cli(context='{"title":"Test Title", "text":"Test Text"}', webhook=True)

In [ ]:
# |export
@call_parse
def cli_batch(
    task_file: str = None,  # task file to run a batch, used exclusively of other other variables
    config_file: str = None,  # config file to set env vars from
):
    if task_file is not None:
        # Means a task file is provided, first we validate it then it's the same as running 1 task but for each line minus the header
        task_sheet = core.get_samplesheet(task_file)

    for row in task_sheet:
        # For each row in the task sheet run the task
        cli(
            context=row["context"],
            webhook=row["webhook"],
            email=row["email"],
            logfile=row["logfile"],
            config_file=config_file,
        )
    return True

In [ ]:
# | hide
from nbdev import nbdev_export

nbdev_export()